<a href="https://colab.research.google.com/github/sihyeon3523/Six_of_cells/blob/scraping/Lyrics_03_SentimentTagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

# 0. Install & import libraries 

In [8]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2 
print(tensorflow.__version__) # 2.8.0  
print(transformers.__version__) # 4.8.2  
print(tensorflow_addons.__version__) # 0.16.1 

import sentencepiece

import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm
import ast  # str list to list

1.0.2
2.8.0
4.8.2
0.17.0


In [9]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [10]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# Import Google Drive 

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
os.getcwd()

'/content/drive/My Drive/proj_fin_seunguk'

In [13]:
# 경로만 바꾸면 됨!
path = '/content/drive/MyDrive/proj_fin_seunguk'# google drive project path
data_path = 'colab_data/temp_data/' # model data path
checkpoint_path = 'colab_data/temp_data/saved_models/' # saved model path

os.chdir(path)

# 1. Load the Data

In [14]:
df_allsongs_original = pd.read_csv('left_150.csv', lineterminator='\n')

In [15]:
df_allsongs_original.head(3)

,Unnamed: 0,Title,Artist,Date,Genre,preprocess_Lyric_ver3,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,\r
0,51,서툰 이별을 하려해,문별 (마마무),2022.03.28,발라드,"['우리 이제 헤어지기로 해', '너도 알잖아', '우린 충분했다는 걸', '이별이...",,,,,,\r
1,52,월급은 통장을 스칠 뿐,스텔라장 (Stella Jang),2017.04.30,"인디음악, 포크/블루스","['나나나 나나나 나나나 ', '나나나나나나 나나나나 ', '나나나 나나나 나나나 ...",,,,,,\r
2,53,"날 봐, 귀순",대성,2008.06.16,성인가요/트로트,"['날 봐 날 봐 귀순 날 봐 날 봐 귀순_x000D_', '날 봐 날 봐 지금 당...",,,,,,\r


In [16]:
df_allsongs = df_allsongs_original[["Title", "Artist", "Date", "Genre", "preprocess_Lyric_ver3"]]
df_allsongs

,Title,Artist,Date,Genre,preprocess_Lyric_ver3
0,서툰 이별을 하려해,문별 (마마무),2022.03.28,발라드,"['우리 이제 헤어지기로 해', '너도 알잖아', '우린 충분했다는 걸', '이별이..."
1,월급은 통장을 스칠 뿐,스텔라장 (Stella Jang),2017.04.30,"인디음악, 포크/블루스","['나나나 나나나 나나나 ', '나나나나나나 나나나나 ', '나나나 나나나 나나나 ..."
2,"날 봐, 귀순",대성,2008.06.16,성인가요/트로트,"['날 봐 날 봐 귀순 날 봐 날 봐 귀순_x000D_', '날 봐 날 봐 지금 당..."
3,"내일이 오면 (Feat. 기리보이, BIG Naughty (서동현))",릴보이 (lIlBOI),2020.12.05,랩/힙합,"['See I just wanna', 'share the vibe with ya',..."
4,치맛바람 (Chi Mat Ba Ram),브레이브걸스,2021.06.17,댄스,"['You know what it is', 'B. girls are back, Br..."
...,...,...,...,...,...
206,일편단심,금잔디,2009.11.30,성인가요/트로트,"['', ' ', '나도 몰래 사랑했나봐 ', '아프도록 사랑했나봐 ', '시간 흐..."
207,Best Of Me,방탄소년단,2017.09.18,랩/힙합,"['When you say that you love me', '난 하늘 위를 걷네'..."
208,화(火花),(여자)아이들,2021.01.11,댄스,"['차디찬 한겨울이 덮친 듯', '시간은 다 얼어버리고', '잔인한 그 바람이 남긴..."
209,태클을 걸지마,진성,2005.12.25,성인가요/트로트,"['지금부터 뛰어', '앞만 보고 뛰어', '', '어떻게 살았냐고 묻지를 마라',..."


In [17]:
print(df_allsongs.info())
print(df_allsongs.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Title                  211 non-null    object
 1   Artist                 211 non-null    object
 2   Date                   211 non-null    object
 3   Genre                  211 non-null    object
 4   preprocess_Lyric_ver3  211 non-null    object
dtypes: object(5)
memory usage: 8.4+ KB
None
       Title Artist        Date Genre  \
count    211    211         211   211   
unique   209    152         198    22   
top       고백    임영웅  2020.12.26  랩/힙합   
freq       2      8           3    34   

                                    preprocess_Lyric_ver3  
count                                                 211  
unique                                                211  
top     ['우리 이제 헤어지기로 해', '너도 알잖아', '우린 충분했다는 걸', '이별이...  
freq                                            

In [18]:
df_allsongs.preprocess_Lyric_ver3.iloc[12]

'["woh I\'m on air", \'구름 위 걸어 다녀 매일\', \'hunnit dolla bills in my pocket\', \'허나 발걸음만은 가볍게\', \'5년 전과는 다른 위치\', \'다른 state of mind\', \'잃을 게 많아 보이긴 하지만 \', "사실 i don\'t care", \'aye say hello to my little \', \'friend like woh\', "시간이 지나도 I\'m on it on it", \'굴린 적 없지 내 잔머린 머리\', \'꽤 오래 자리를 비웠었지만\', \'너도 알잖아 나의 랩은 iconic\', \'그레이 형의 BEAT \', \'이건 family business\', \'불을 더 지피지 we gotta keep it real\', \'다시 뭉쳤지 Team AOMG\', \'커져버린 덩치 think this is the one\', \'어느새 여기까지 이제야 보여 멀리까지\', \'정확히 기억하지\', \'끝나고 집에 돌아가던 날이\', \'오늘은 점을 찍고\', \'제대로 돌아가려 하지\', \'same team same squad \', \'but different day\', \'바뀐 게 있다면 직접 보길 원해\', \'새로운 역사를 난 이뤄내지\', \'so come see me live now\', \'We on air\', "You\'re bout to watch me dance", \'올라가지 We go way up\', \'구름 위에서 난 헤엄\', \'시간이 들었지만 \', \'어차피 우승은 내 말마따나 음\', \'위험해 위험해 생방 \', \'불구덩이 내 자리 맡아놔\', \'5년 전에 그린 TV 속의 팀\', \'돌아왔다 각 풀어진 채로 \', \'막 수경으로 만기 \', \'다시 I can be your party band \', \'In everywhere 난 groovy \', \'아직도 걔네들은 멍청하더라 \', \'꼭 이래 보여줘야 불식 \', \'

# Colab 끊겨서 뒷부분 장르 태깅 안되는 경우 방지 all rows shuffle

In [19]:
df_allsongs = df_allsongs.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset

# Load saved Model


In [20]:
# 1) Load the Model-builder (function)
with open(data_path + 'model_BERTfunction_v1.pkl', 'rb') as f:
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
with open(data_path + 'tokenizer-bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f) 


# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver) 
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 

model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

# Classifier as a function

In [21]:
def predict_sentiment_element(sentence, tokenizer, model):
    """
    가사 내 문장 별 확률 값 추출
    """    
    SEQ_LEN = 300 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence), # 특수문자 제거
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    pred = model.predict(new_inputs) 
    all_pred_proba = np.round(pred * 100, 2)
    
    #top_pred_proba = np.round(np.max(pred) * 100, 2) 
    #pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(pred, axis=1)[0]] # 수정하면 4개 클래스 다 나올듯! 
    #print("{}% 확률로 {} 텍스트입니다.".format(top_pred_proba, pred_class))

    return all_pred_proba 

In [22]:
def predict_sentiment_lyrics(lyrics):
    """
    가사의 감성 확률 값 추출, 문장별 감성 확률의 정규화 평균  
    가사내 문장 단위로 predict_sentiment_element 함수 적용   
    마지막 원소 '' 빈칸임. <br/> 태그 기준으로 split해서 나옴, 빈칸인 element 제거
    """
    lyrics = ast.literal_eval(lyrics) # str list to list
    lyrics = list(filter(lambda x: x != '', lyrics))

    print(len(lyrics)) 
    print(lyrics)


    for i in range(len(lyrics)):
        
        if lyrics[i] != '':
            all_pred_proba = predict_sentiment_element(lyrics[i], tokenizer, model)
            anger = all_pred_proba[0][0]
            scary = all_pred_proba[0][1]
            sad = all_pred_proba[0][2]
            happy = all_pred_proba[0][3]
            #print(i, lyrics[i], all_pred_proba) 

        anger += anger 
        scary += scary
        sad  += sad
        happy += happy

    total_array = np.ndarray.round( (np.array([(anger, scary, sad, happy)]) / (anger + scary + sad + happy)), 2)
    top_pred_prob = total_array[0][np.argmax(total_array, axis=1)[0]]
    top_pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(total_array, axis=1)[0]]

    return total_array, top_pred_class, top_pred_prob

In [23]:
# test_list = df_allsongs.preprocess_Lyric_ver3.iloc[35]
# total_array, top_pred_class, top_pred_prob = predict_sentiment_lyrics(test_list)

In [24]:
# print(total_array) 
# print(top_pred_class)  
# print(top_pred_prob)  

preprocess_Lyric_ver3에 적용 <br> 

list가 str 형태로 row에 저장되었기 때문에 ast library로 다시 list로 바꿔주어야한다.<br>
(str list to list)

# Apply model

In [25]:
# Emotion = []
# Probability = []
# 분노혐오 = []
# 놀람공포 = []
# 슬픔 = []
# 행복 = []


# for row in tqdm(df_allsongs.iterrows()):


#         total_array, top_pred_class, top_pred_prob = predict_sentiment_lyrics(row[1][-1])
#         Emotion.append(top_pred_class)
#         Probability.append(top_pred_prob)
#         분노혐오.append(total_array[0][0])
#         놀람공포.append(total_array[0][1])
#         슬픔.append(total_array[0][2])
#         행복.append(total_array[0][3]) 

# df_allsongs['Emotion'] = Emotion
# df_allsongs['Proba'] = Probability
# df_allsongs['분노혐오'] = 분노혐오
# df_allsongs['놀람공포'] = 놀람공포
# df_allsongs['슬픔'] = 슬픔
# df_allsongs['행복'] = 행복

# df_allsongs.to_csv('songtagged_may27.csv')

In [26]:
df_allsongs['Emotion'] = " "
df_allsongs['Probability'] = " "
df_allsongs['분노혐오'] = " "
df_allsongs['놀람공포'] = " "
df_allsongs['슬픔'] = " "
df_allsongs['행복'] = " "

for i in tqdm(df_allsongs.index):
    lyrics = df_allsongs['preprocess_Lyric_ver3'].iloc[i]
    total_array, top_pred_class, top_pred_prob = predict_sentiment_lyrics(lyrics)
    df_allsongs['Emotion'].iloc[i] = top_pred_class 
    df_allsongs['Probability'].iloc[i] = top_pred_prob
    df_allsongs['분노혐오'].iloc[i] = total_array[0][0]
    df_allsongs['놀람공포'].iloc[i] = total_array[0][1]
    df_allsongs['슬픔'].iloc[i] = total_array[0][2]
    df_allsongs['행복'].iloc[i] = total_array[0][3]

    df_allsongs.to_csv('songtagged_may29_left.csv')

clear_output()

  0%|          | 0/211 [00:00<?, ?it/s]

37
['Stay 내 눈물이 마를 때까지', 'Stay 내가 나를 모를 때까지', 'Stay 아주 조금만 기다려', 'Stay 내 기억의 주인은 나야', 'Stay 내가 널 보내줄 때까지', 'Stay 내 기억속에서라도', '조금의 따뜻함이라도 ', '간직할 수 있게해 줘', '난 이미 얼어버릴 듯 ', '한없이 차가워', '너마저 떠나면 나에겐 ', '이제 아름다움이 없어', '난 이미 버려져 있고 ', '한없이 더러워', 'Hey 이미 꽤 오랜 시간동안 ', '내 안에 머물러 있었잖아', '이제 그냥 집이라고 생각해', '조금의 따뜻함이라도 ', '간직할 수 있게해줘', '난 이미 얼어버릴 듯 ', '한없이 차가워', '너마저 떠나면 나에겐 ', '이제 아름다움이 없어', '난 이미 버려져 있고 ', '한없이 더러워', 'Stay inside my dear', "Don't you come out my dear ", '조금의 따뜻함이라도', '간직할 수 있게 해줘', '난 이미 얼어버릴 듯 ', '한없이 차가워', '너 마저 떠나면 나에겐', '이제 아름다움이 없어', '난 이미 죽어 버릴 듯 ', '한없이 더러워', 'Stay my dear', 'Stay my dear']


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>]
  0%|          | 1/211 [00:54<3:10:58, 54.56s/it]

28
['당신은 내게 그런 사람이에요', '늘 가슴에 품고 살아요', '당신에게서 나는 어떤 사람일까', '가끔은 생각을 해줄까', '별처럼 반짝이던', '우리의 시간도 지나갔죠', '꿈처럼 다시 볼 수는 없나요', '바람처럼 스쳐만 가나요', '사랑아 내 사랑아 나를 떠나지 말아', '다시 만날 수는 없는 내 사랑아', '내 맘속에 별이 되어', '언제라도 그렇게', '영원히 빛나고 있어줘', '그리워 지새우던', '우리의 시간도 지나갔죠', '꿈처럼 다시 볼 수는 없나요', '구름처럼 흘러만 가나요', '사랑아 내 사랑아 나를 떠나지 말아', '다시 만날 수는 없는 내 사랑아', '내 맘속에 별이 되어', '언제라도 그렇게', '영원히 빛나고 있어줘', '사랑아 내 사랑아 나를 잊지 말아줘', '기다려도 오지 않을 사랑아', '내 맘속에 별이 되어', '언제라도 그렇게', '영원히 빛나고 있어줘', '사랑아 나의 사랑아']


  1%|          | 2/211 [01:10<1:50:04, 31.60s/it]

...

 99%|█████████▉| 209/211 [1:35:13<01:27, 43.77s/it]

88
['밤이 되면 어김없이 ', '찾아오는 이 느낌이 ', '왠지 원망스러워 ', '괜히 또 전화 걸어 봐 ', '너도 나와 같은 마음일까 봐 ', '나 지금 센치해 ', '천장만 바라봐 ', '아무것도 없는 바탕에 ', '그림 그려봤자 ', '나 조금 센치해 ', '좁은 내 방 한 칸 ', '오늘따라 너무 넓어 ', '혼자 누워있기엔 ', 'Bed time story ', '불러줘 자장가 ', '잠들고 싶어도 ', '양들이 멀리 도망가 ', '창문 부딪히는 소리가', '천둥인 듯이', '무서워 오늘 밤', '종이 울려 밤 열두시', '달은 내게 인사해도', '오늘은 별로 반갑지 않아', '머릴 흔들면 어지러워', '내 곁에 아무도 없잖아', '감정은 low한데', '난 우주까지 가고파', '이 맘 누가 알아줄까', '죽고 싶어 말로만', '밤이 되면 어김없이', '찾아오는 이 느낌이', '왠지 원망스러워', '괜히 또 전화 걸어 봐', '너도 나와 같은 마음일까 봐', '나 지금 센치해', '천장만 바라봐', '아무것도 없는 바탕에', '그림 그려봤자', '나 조금 센치해', '좁은 내 방 한 칸', '오늘따라 너무 넓어', '혼자 누워있기엔', '그냥 하늘에 떠 있는 별이 되고파', '아무것도 안 해도', '스스로 빛을 내잖아', '어두워진 밤과 같이', '어두워진 마음', '외로워 위태로워', '누가 날 알아줄까', '괜히 어울리지 않는 재즈', '음악을 틀어요 레이찰스같이', '비가 나를 적시듯 ', '음악 소린 울려 퍼져', '귀에서 발끝까지', '조심해 조심해 SNS', '감성에 취해 헛소리 적지 않기', '혈기 왕성한 마음에', '전 여자친구한테 전화 않기', '밤이 되면 어김없이', '찾아오는 이 느낌이', '왠지 원망스러워', '괜히 또 전화 걸어 봐', '너도 나와 같은 마음일까 봐', '나 지금 센치해', '천장만 바라봐', '아무것도 없는 바탕에', '그림 그려봤자', '나 조금 센치해', '좁은 내 방 한 칸', '오늘따라 너무 넓어'

100%|█████████▉| 210/211 [1:36:00<00:44, 44.88s/it]

16
['기억하나요 가슴 아픈 사연을', '내 님 오실 날을 저울질 하나요', '한참 후에야 그 마음을 알았죠', '내가 아닌 곳에 머물러 있다는 걸', '내게 올 순 없나요 사랑할 순 없었나요', '그대 헤일 수 없는 맘 나였던가요', '잊지 말아요 가슴 아픈 사랑이', '슬퍼하는 날엔 내가 서 있을게요', '내게 올 순 없나요 사랑할 순 없었나요', '그대 헤일 수 없는 맘 나였던가요', '이대로 안되나요 돌아올 길 잊었나요', '그대 헤일 수 없는 맘 나였던가요', '잊지 말아요 가슴 아픈 사랑이', '슬퍼하는 날엔 내가 서 있을게요', '잊지 말아요 가슴 아픈 사랑이', '슬퍼하는 날엔 내가 서 있을게요']


100%|██████████| 211/211 [1:36:09<00:00, 27.34s/it]
